Classify images into 10 categories:
airplane, car, bird, cat, deer, dog, frog, horse, ship, truck"

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models

c:\Users\pc\miniconda3\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

c:\Users\pc\miniconda3\Lib\site-packages\keras\src\datasets\cifar.py:18: VisibleDeprecationWarning: dtype(): align should be passed as Python or NumPy boolean but got `align=0`. Did you mean to pass a tuple to create a subarray type? (Deprecated NumPy 2.4)
  d = cPickle.load(f, encoding="bytes")


Each image is 32×32×3

32 → width

32 → height

3 → RGB (Red, Green, Blue)

Labels are numbers from 0 to 9

In [3]:
x_train = x_train / 255.0
x_test = x_test / 255.0

Original pixel values: 0 – 255

After dividing: 0 – 1

Helps:

Faster learning

Stable gradients

Better accuracy

👉 CNNs always prefer small, consistent values.

In [4]:
model = models.Sequential([
    
    # Convolution Block 1
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(32,32,3)),
    #32 filters → model learns 32 different features
    #3×3 filter size → looks at small image regions
    #Slides over image → detects edges, colors, textures
    #Same filter used everywhere → weight sharing
    #ReLU → removes useless negative values
    #This is where CNN actually “sees” the image.

    layers.MaxPooling2D((2,2)),
    #Takes maximum value from each 2×2 region
    #Reduces image size
    #Keeps strongest features
    #Like summarizing important information.
    
    # Convolution Block 2
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    
    # Flatten and Dense Layers
    layers.Flatten(),
    #Converts feature maps into 1D vector
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])


c:\Users\pc\miniconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [6]:
model.fit(x_train, y_train, epochs=10, batch_size=64)

Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 25s 27ms/step - accuracy: 0.4415 - loss: 1.5446
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 19s 24ms/step - accuracy: 0.5722 - loss: 1.2146
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 18s 23ms/step - accuracy: 0.6252 - loss: 1.0740
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 19s 24ms/step - accuracy: 0.6604 - loss: 0.9824
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 20s 23ms/step - accuracy: 0.6809 - loss: 0.9223
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 21s 24ms/step - accuracy: 0.7006 - loss: 0.8676
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 18s 23ms/step - accuracy: 0.7146 - loss: 0.8256
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 18s 23ms/step - accuracy: 0.7284 - loss: 0.7834
Epoch 9/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 22s 28ms/step - accuracy: 0.7437 - loss: 0.7425
Epoch 10/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 20s 26ms/step - accuracy: 0.7549 - loss: 0.7108


In [7]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print("Test Accuracy:", test_acc)


313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6876 - loss: 0.9326
Test Accuracy: 0.6876000165939331


✅ UPGRADE 1: Make CNN Slightly Deeper

CIFAR-10 images are complex:

colors

textures

overlapping objects

2 Conv layers are not enough to capture this.

So we:

add one more Conv layer

increase feature extraction depth

In [8]:
model = models.Sequential([

    # Block 1
    layers.Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(32,32,3)),
    layers.MaxPooling2D((2,2)),

    # Block 2
    layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    layers.MaxPooling2D((2,2)),

    # Block 3 (NEW)
    layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])


✅ UPGRADE 2: Add Batch Normalization 
During training:

feature values keep changing

learning becomes unstable

Batch Normalization:

keeps values in a stable range

allows faster & deeper learning

reduces vanishing/exploding gradients

👉 Almost all modern CNNs use it.

In [9]:
model = models.Sequential([

    layers.Conv2D(32, (3,3), padding='same', input_shape=(32,32,3)),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), padding='same'),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128, (3,3), padding='same'),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])


✅ UPGRADE 3: Add Dropout + Train Longer (Generalization)

CNN is now powerful → risk of overfitting

Dropout:

forces model to not depend on single neurons

improves test accuracy

Also:

CIFAR-10 needs more epochs

In [10]:
model = models.Sequential([

    layers.Conv2D(32, (3,3), padding='same', input_shape=(32,32,3)),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), padding='same'),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128, (3,3), padding='same'),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])


In [11]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(
    x_train, y_train,
    epochs=30,
    batch_size=64,
    validation_data=(x_test, y_test)
)


Epoch 1/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 84s 100ms/step - accuracy: 0.4101 - loss: 1.6222 - val_accuracy: 0.5201 - val_loss: 1.3123
Epoch 2/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 81s 104ms/step - accuracy: 0.5393 - loss: 1.2673 - val_accuracy: 0.6130 - val_loss: 1.1125
Epoch 3/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 77s 98ms/step - accuracy: 0.5967 - loss: 1.1310 - val_accuracy: 0.5722 - val_loss: 1.1764
Epoch 4/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 91s 116ms/step - accuracy: 0.6327 - loss: 1.0368 - val_accuracy: 0.6756 - val_loss: 0.9428
Epoch 5/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 92s 117ms/step - accuracy: 0.6572 - loss: 0.9697 - val_accuracy: 0.6647 - val_loss: 0.9794
Epoch 6/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 87s 111ms/step - accuracy: 0.6819 - loss: 0.9041 - val_accuracy: 0.6143 - val_loss: 1.2478
Epoch 7/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 90s 115ms/step - accuracy: 0.7026 - loss: 0.8512 - val_accuracy: 0.6640 - val_loss: 0.9735
Epoch 8/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 81s 104ms/step - accuracy: 0.7108 - loss: 0.

In [12]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print("Test Accuracy:", test_acc)


313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.7451 - loss: 1.1145
Test Accuracy: 0.7451000213623047
